# Test 2
***

### Выполнил Федотов Р. А., 531 гр.

We are going to work with the following dataset: fluid current in a tube.
Some statistics are collected for dataset, incl. mean, skewness, kurtosis, etc. We are predicting flow rate ('tohn/hour'). We need to build confidence and predictive intervals.

In [1]:
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model, preprocessing, model_selection
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
import statsmodels.stats.api as sms

In [2]:
df = pd.read_csv('exxsol_data.csv', sep=';', header=(0))

There are 10 features and 1 label to predict:

In [3]:
print(df.columns.values)

['mean' 'std' 'skew' 'kurt' 'RMS' 'crest' 'freq_peak' 'shan' 'perm' 'temp'
 'tohn/hour']


In [4]:
df

,mean,std,skew,kurt,RMS,crest,freq_peak,shan,perm,temp,tohn/hour
0,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
1,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
2,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
3,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
4,0.031998,0.972590,-0.056116,-1.546518,0.973106,0.921600,256.00512,9.020444,0.973701,21.14,3.56
...,...,...,...,...,...,...,...,...,...,...,...
695,0.020624,0.458015,0.135061,-0.499570,0.458475,0.819037,8.00016,13.284682,1.220259,24.10,0.56
696,-0.001068,0.628883,0.098332,-0.610439,0.628878,0.845106,6.00012,12.998447,1.174423,24.10,0.56
697,-0.001068,0.628883,0.098332,-0.610439,0.628878,0.845106,6.00012,12.998447,1.174423,24.10,0.56
698,-0.050864,0.558909,-0.025527,-0.160860,0.561213,0.828625,6.00012,13.042021,1.196795,24.10,0.56


In [5]:
y = df['tohn/hour']
freq_temp = df[['freq_peak','temp']]

Physics tells us that flow rate is a function of a frequency peak and temperature.

In [6]:
freq_temp, y = shuffle(freq_temp, y)

In [51]:
freq_temp, y = shuffle(freq_temp, y)

# split data into training and testing sets
#from sklearn.model_selection import train_test_split
#train_freq, test_freq, train_y, test_y = train_test_split(freq, y, train_size=0.7, random_state=2)

model_lr = linear_model.LinearRegression()
pred = model_selection.cross_val_predict(model_lr,freq_temp, y.ravel(), cv=20)
scores_r2 = model_selection.cross_val_score(model_lr, freq_temp, y, scoring='r2',cv=20)

In [33]:
model_elastic = linear_model.ElasticNetCV(cv=10, max_iter=3000, n_alphas=10)
pred_elastic = model_selection.cross_val_predict(model_elastic, freq_temp, y.ravel(), cv=20)
scores_r2_elastic = model_selection.cross_val_score(model_elastic, freq_temp, y.ravel(), scoring='r2',cv=20)

## Q0: Evaluation based on predictions. Model selection based on $R^2$ metric. Can we choose any model based on 95% confidence interval?


Для сравнения $R^2$-оценок воспользуемся t-test for mean difference.

Пусть $R^2_1$ - истинное значение $R^2$ для первой модели, $R^2_2$ - истинное значение $R^2$ для второй модели. Предположим, что мы считаем, что вторая модель с регуляризацией лучше, чем первая: например, потому, что она может избежать переобучения. 

Гипотеза:

$H_0: R^2_1 = R^2_2$

$H_1: R^2_1 < R^2_2$

In [47]:
alpha = 0.05

cm = sms.CompareMeans(sms.DescrStatsW(scores_r2), sms.DescrStatsW(scores_r2_elastic))
cm.tconfint_diff(alternative = 'larger', usevar='pooled', alpha = alpha)

(-0.08316724393142731, inf)

Видим, что $0$ попадает в этот доверительный интервал. Следовательно, мы не можем утверждать, что $R^2$ у этих моделей отличаются друг от друга на уровне $\alpha = 0.05$.

## Q1:  Goodness of fit. Implement AIC, AICc and compare models using these criteria

Так как обе модели - регрессии, построенные на $2$ факторах, то в обеих моделях будет $k = 3$ параметра:

In [53]:
def aic_criterion(y_true: np.ndarray, y_pred: np.ndarray, n_params: int) -> float:
    n = y_pred.shape[0]
    rss = np.sum((y_pred - y_true)**2)
    return 2*n_params + n * np.log(rss)

In [63]:
aic_lr = aic_criterion(y_true = y, y_pred = pred, n_params = 3)
print(f'aic for linear model: {aic_lr}')

aic for linear model: 3321.825539884706


In [64]:
aic_elastic = aic_criterion(y_true = y, y_pred = pred_elastic, n_params = 3)
print(f'aic for elastic linear model: {aic_elastic}')

aic for elastic linear model: 4937.453348990894


Видим, что первая модель регрессии более предпочтительна, чем вторая. 

In [68]:
def aic_criterion_c(y_true: np.ndarray, y_pred: np.ndarray, n_params: int) -> float:
    n = y_pred.shape[0]
    rss = np.sum((y_pred - y_true)**2)
    return (n + n_params)/(n - n_params - 2) +  np.log(rss/n)

In [69]:
aic_c_lr = aic_criterion_c(y_true = y, y_pred = pred, n_params = 3)
print(f'aic_c for linear model: {aic_c_lr}')

aic_c for linear model: -0.802675915269776


In [70]:
aic_c_elastic = aic_criterion_c(y_true = y, y_pred = pred_elastic, n_params = 3)
print(f'aic_c for elastic linear model: {aic_c_elastic}')

aic_c for elastic linear model: 1.505363812024779


Видим, что первая модель регрессии более предпочтительна, чем вторая. 

## Q2:  More advanced. Implement Mallow’s Cₚ criterion:

Wiki defenition of Mallows $C_p$:

$C_{p}={\frac {1}{n}}(\operatorname{RSS} +2p{\hat\sigma}^{2})$

In [75]:
def mallow_c(y_true: np.ndarray, y_pred: np.ndarray, n_params: int) -> float:
    n = y_pred.shape[0]
    rss = np.sum((y_pred - y_true)**2)
    sigma_est = y_pred.std()
    return (1/n)*(rss + 2*(n_params - 1)*sigma_est**2)

In [76]:
mallow = mallow_c(y_true = y, y_pred = pred, n_params = 3)
print(f'mallow for linear model: {mallow}')

mallow for linear model: 0.16772518409905626


In [77]:
mallow_elastic = mallow_c(y_true = y, y_pred = pred_elastic, n_params = 3)
print(f'mallow for elastic linear model: {mallow_elastic}')

mallow for elastic linear model: 1.6428658492187092


## Q3: There is also BIC criteria. Can you recommend when to use AIC vs BIC and vice versa?


AIC и BIC предназначены для разных вещей.

AIC в основном подходит для ситуации, когда не обязательно предполагается, что существует «модель», а есть набор эффектов разного размера, и мы бы хотели получить хорошую ошибку прогнозов. Таким образом, по мере увеличения размера выборки выбор модели AIC также расширяется, поскольку становятся актуальными все меньшие и меньшие эффекты (в том смысле, что их включение в среднем лучше, чем их исключение).

BIC, с другой стороны, в основном предполагает, что модель находится в наборе кандидатов, и необходимо ее найти. BIC имеет тенденцию выделять одну модель по мере роста числа наблюдений, AIC этого не делает. В результате, при увеличении размера выборки, AIC имеет тенденцию выбирать несколько более параметрически большие модели, чем BIC. 